<a href="https://colab.research.google.com/github/AstikaNehra/GrammarChecker/blob/master/Grammar_Checker_3_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
url_train = 'https://raw.githubusercontent.com/AstikaNehra/GrammarChecker/master/final_train.csv'
url_test = 'https://raw.githubusercontent.com/AstikaNehra/GrammarChecker/master/final_test.csv'
df_train = pd.read_csv(url_train,error_bad_lines=False, encoding='cp1252',header=None)
df_test = pd.read_csv(url_test,error_bad_lines=False, encoding='cp1252',header=None)

In [ ]:
df_train

,0,1
0,One more pseudo generalization and I'm giving up.,1
1,One more pseudo generalization or I'm giving up.,1
2,The more we study verbs the crazier they get.,1
3,Day by day the facts are getting murkier.,1
4,I'll fix you a drink.,1
...,...,...
4785,My name is Astika,1
4786,I am Astika,1
4787,I is Astika,0
4788,I took your umbrella,1


In [ ]:
data_train=df_train.values
data_test=df_test.values

In [ ]:
X_train = data_train[:,0]
Y_train = data_train[:,1]
X_test = data_test[:,0]
Y_test = data_test[:,1]

# Learn Word Embedding

In [ ]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

tokenizer_obj=Tokenizer()
total_reviews=X_train+X_test
tokenizer_obj.fit_on_texts(total_reviews)

#pad sequences
max_length=max([len(s.split())for s in total_reviews])

#define vocabulary size
vocab_size=len(tokenizer_obj.word_index)+1

X_train_tokens=tokenizer_obj.texts_to_sequences(X_train)
X_test_tokens=tokenizer_obj.texts_to_sequences(X_test)

X_train_pad=pad_sequences(X_train_tokens,maxlen=max_length,padding='post')
X_test_pad=pad_sequences(X_test_tokens,maxlen=max_length,padding='post')

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU
from keras.layers.embeddings import Embedding

EMBEDDING_DIM=100

print("Building Model...")

model=Sequential()
model.add(Embedding(vocab_size,EMBEDDING_DIM, input_length=max_length))
model.add(GRU(units=32,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

Using TensorFlow backend.


Building Model...


In [ ]:
print('Training...')

model.fit(X_train_pad,Y_train,batch_size=128,epochs=25,validation_data=(X_test_pad,Y_test),verbose=2)

Training...


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 4790 samples, validate on 4790 samples
Epoch 1/25
 - 3s - loss: 0.6215 - accuracy: 0.7004 - val_loss: 0.6102 - val_accuracy: 0.7025
Epoch 2/25
 - 2s - loss: 0.6118 - accuracy: 0.7023 - val_loss: 0.6117 - val_accuracy: 0.7025
Epoch 3/25
 - 2s - loss: 0.6103 - accuracy: 0.7023 - val_loss: 0.6124 - val_accuracy: 0.7025
Epoch 4/25
 - 2s - loss: 0.6096 - accuracy: 0.7023 - val_loss: 0.6098 - val_accuracy: 0.7025
Epoch 5/25
 - 2s - loss: 0.6091 - accuracy: 0.7023 - val_loss: 0.6095 - val_accuracy: 0.7025
Epoch 6/25
 - 2s - loss: 0.6087 - accuracy: 0.7023 - val_loss: 0.6122 - val_accuracy: 0.7025
Epoch 7/25
 - 2s - loss: 0.6096 - accuracy: 0.7023 - val_loss: 0.6110 - val_accuracy: 0.7025
Epoch 8/25
 - 2s - loss: 0.6107 - accuracy: 0.7023 - val_loss: 0.6121 - val_accuracy: 0.7025
Epoch 9/25
 - 2s - loss: 0.6094 - accuracy: 0.7023 - val_loss: 0.6115 - val_accuracy: 0.7025
Epoch 10/25
 - 2s - loss: 0.6106 - accuracy: 0.7023 - val_loss: 0.6095 - val_accuracy: 0.7025
Epoch 11/25
 - 2s - l

In [ ]:
test_samples=["Hi how are you", "Hi how you", "I goes there"]
test_samples_tokens=tokenizer_obj.texts_to_sequences(test_samples)
test_samples_tokens_pad=pad_sequences(test_samples_tokens,maxlen=max_length)

model.predict(x=test_samples_tokens_pad)

array([[0.5659108],
       [0.5722165],
       [0.5950344]], dtype=float32)